# Kaplan-Meier Estimation

[Run this notebook on Colab](https://colab.research.google.com/github/AllenDowney/SurvivalAnalysisPython/blob/master/02_kaplan_meier.ipynb)

This notebook introduces Kaplan-Meier estimation, a way to estimate a hazard function when the dataset includes both complete and incomplete cases.
To demonstrate, I'll use a small set of hypothetical data.  

## Dog adoption data

Suppose you are investigating the time it takes for dogs to get adopted from a shelter.  You visit a shelter every week for 10 weeks, and record the arrival time for each dog and the adoption time for each dog that was adopted.

Here's what the data might look like.  

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
obs = pd.DataFrame()

obs['start'] = 0,1,2,2,4,6,7
obs['end'] = 5,2,6,9,9,8,9
obs['status'] = 1,1,1,0,0,1,0

obs

This `DataFrame` contains one row for each dog and three columns:

* `start`: arrival time, in weeks since the beginning of the study

* `end`: adoption date, for dogs that were adopted, or `9` for dogs that had not been adopted at the end of the study

* `status`: `1` for dogs that were adopted; `0` for dogs that were not.

## Plotting lifelines

The following function visualizes the data.

In [ ]:
def plot_lifelines(obs):
    """Plot a line for each observation.
    
    obs: DataFrame
    """
    for y, row in obs.iterrows():
        start = row['start']
        end = row['end']
        status = row['status']
        
        if status == 0:
            # ongoing
            plt.hlines(y, start, end, color='C0')
        else:
            # complete
            plt.hlines(y, start, end, color='C1')
            plt.plot(end, y, marker='o', color='C1')
            
    plt.xlabel('Time (weeks)')
    plt.ylabel('Dog index')
    plt.gca().invert_yaxis()

Here are the results:

In [ ]:
plot_lifelines(obs)

Each line represents the time a dog spends at the shelter. Each dot represents an adoption.
We can see, for example:

* The dog with index 0 arrived during week 0, and was adopted during week 5.

* The dog with index 3 arrived during week 2, and had not been adopted at the end of week 9.


## Estimating survival

Now suppose we want to know the distribution of "survival time" from arrival to adoption.
For the dogs that were adopted, we have all the data we need.  
For the others, we have only partial information: if a dog hasn't been adopted yet, we don't know when it will be, but we can put a lower bound on it.

When we have a mixture of complete and incomplete observations -- adopted and unadopted dogs -- we can't compute the Survival function directly.
Instead, we have to work backwards: we estimate the hazard function first, then use it to compute the survival function, CDF, and PMF.

Specifically, we'll use Kaplan-Meier estimation, which is based on two key ideas.

The first idea is that we can ignore the arrival time in the observed data, and consider only the durations.  In effect, we can take the actual lifelines and shift them so they all start at 0, like this:

In [ ]:
duration = obs['end'] - obs['start']

In [ ]:
shifted = obs.copy()
shifted['start'] = 0
shifted['end'] = duration

In [ ]:
plot_lifelines(shifted)
plt.xlabel('Duration (weeks)');

Notice that the x-axis in this figure is duration, not time.

The second key idea is that we can estimate the hazard function by considering:

* The number of dogs adopted at each duration, divided by

* The number of dogs "at risk" at each duration, where "at risk" means that they *could* be adopted.

For example:

* At duration 1, there is 1 adoption out of 7 dogs at risk, so the hazard rate is `1/7`.

* At duration 2, there is 1 adoption out of 6 dogs at risk, so the hazard rate is `1/6`.

* At duration 4, there is 1 adoption out of 4 dogs at risk, so the hazard rate is `1/4`.

And so on. Now let's see how that works computationally.

## Estimating the hazard function

First I'll select the durations of the complete and ongoing cases.

In [ ]:
complete = duration[obs['status']==1]
complete

In [ ]:
ongoing = duration[obs['status']==0]
ongoing

Now I'll compute the PMF for complete and ongoing cases: 

In [ ]:
def make_pmf(sample):
    return pd.Series(sample).value_counts().sort_index()

In [ ]:
pmf_complete = make_pmf(complete)
pmf_complete

In [ ]:
pmf_ongoing = make_pmf(ongoing)
pmf_ongoing

Here's what the PMFs look like in a table.

In [ ]:
d = dict(pmf_complete=pmf_complete, 
         pmf_ongoing=pmf_ongoing)

In [ ]:
index = pmf_complete.index.union(pmf_ongoing.index)

In [ ]:
df = pd.DataFrame(d, index=index).fillna(0)
df

This table indicates that one dog was adopted at 1 week, another dog was adopted at 2 weeks, and a third dog was observed, unadopted, having been in the shelter for 2 weeks.

We can use the PMFs to compute survival functions for the two groups:

In [ ]:
def make_surv(pmf):
    cdf = pmf.cumsum()
    return pmf.sum() - cdf

In [ ]:
df['s_complete'] = make_surv(df['pmf_complete'])
df['s_ongoing'] = make_surv(df['pmf_ongoing'])
df

The first row of this table indicates that 3 of the adopted dogs and 3 of the unadopted dogs were in the shelter for more than 1 week.

We can use this table to compute the number of dogs "at risk" at each duration, `t`, which is the sum of:

1) The number cases that end at `t`, which is `pmf_complete`.

2) The number of ongoing cases observed at `t`, which is `pmf_ongoing`,

3) The number of complete cases that survive past `t`, which is `s_complete`.

4) The number of ongoing cases that survive past `t`, which is `s_complete`.

Here's what that looks like.

In [ ]:
df['at_risk'] = df.sum(axis=1)
df



Finally, the hazard function is the ratio of `pmf_complete` and `at_risk`:

In [ ]:
df['hazard'] = df['pmf_complete'] / df['at_risk']
df

## Working backwards

Given the hazard function, we can work backwards to compute the survival curve.

The hazard function is the probability of being adopted at each duration, so its complement is the probability of *not* being adopted. 

In order to survive past `t`, a dog has to *not* be adopted at all durations up to and including `t`.

So the survival function is the cumulative product of the complement of the hazard function.

In [ ]:
df['surv'] = (1 - df['hazard']).cumprod()
df

The CDF is the complement of the survival function.

In [ ]:
df['cdf'] = 1 - df['surv']
df

And the PMF is the difference between adjacent elements of the CDF.

In [ ]:
df['pmf'] = np.diff(df['cdf'], prepend=0)
df

## lifelines

Kaplan-Meier estimation is available in a library called `lifelines`.
First I'll import it and create a `KaplanMeierFitter`.

In [ ]:
# If we're running in Colab, install lifelines

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install lifelines

In [ ]:
from lifelines import KaplanMeierFitter
kmf = KaplanMeierFitter()

Now we need two sequences, the durations, including complete and ongoing cases.

In [ ]:
T = shifted['end']
T

And an event flag that indicates whether a case is complete.

In [ ]:
E = shifted['status']

The `fit` method does the Kaplan-Meier estimation.

In [ ]:
kmf.fit(T, E)

Now the `Fitter` object contains the estimated survival function.

In [ ]:
kmf.survival_function_

`timelines` includes an element at `t=0`, but other than that it is identical to what we computed (except for floating-point error).

In [ ]:
max(abs(kmf.survival_function_['KM_estimate'] - df['surv']).dropna())

`lifelines` also computes a confidence interval for the survival function.

In [ ]:
ci = kmf.confidence_interval_survival_function_
ci

With such a small dataset, the CI is pretty wide.

In [ ]:
ts = ci.index
low, high = np.transpose(ci.values)

plt.fill_between(ts, low, high, color='gray', alpha=0.3)
kmf.survival_function_.plot(ax=plt.gca())
plt.ylabel('Survival function');

Part of [Survival Analysis in Python](https://allendowney.github.io/SurvivalAnalysisPython/)

Allen B. Downey

[Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/)